# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install matplotlib

In [2]:
!pip install gmaps

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys2 import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
vacationpy_df = pd.read_csv("cities_.csv")
vacationpy_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Poum,-20.2333,164.0167,79.21,77,100,21.43,NC,1616169771
1,Port,-7.0349,29.7638,77.38,69,89,1.90,CD,1616169771
2,Luanda,-8.8368,13.2343,86.00,70,20,13.80,AO,1616169772
3,Alofi,-19.0595,-169.9187,75.20,100,100,3.44,NU,1616169772
4,Bethel,41.3712,-73.4140,35.60,32,1,16.11,US,1616169773
...,...,...,...,...,...,...,...,...,...
553,Camacha,33.0833,-16.3333,62.60,55,75,13.80,PT,1616169939
554,Petkovci,44.4531,19.0761,42.80,49,40,3.44,BA,1616169940
555,Amvrosiyivka,47.7935,38.4777,39.34,72,100,5.86,UA,1616169727
556,Mattru,7.6244,-11.8332,96.40,37,25,5.91,SL,1616169941


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

#Lat and lng locations.
locations = vacationpy_df[["Lat", "Lng"]]

#Humidity as weight.
humidity = vacationpy_df["Humidity"]

#Heatmap.
fig = gmaps.figure() #center=(46.0, -5.0), zoom_level=2
max_intensity = np.max(humidity)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [18]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#Criteria: max temp lower than 80, higher than 70. Wind speed > 10 mph. Zero cloudiness. Drop rows that dont meet criteria. 
wcriteria_df = vacationpy_df.loc[(vacationpy_df["Wind Speed"] <= 10) & (vacationpy_df["Cloudiness"] == 0) & (vacationpy_df["Max Temp"] >= 70) & (vacationpy_df["Max Temp"] <= 80)].dropna()

wcriteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
91,Pisco,-13.7000,-76.2167,73.99,78,0,8.05,PE,1616169250
154,Chui,-33.6971,-53.4616,71.38,53,0,6.31,UY,1616169282
184,Honāvar,14.2833,74.4500,78.75,82,0,3.80,IN,1616169298
197,Loikaw,19.6742,97.2094,74.64,35,0,3.11,MM,1616169305
219,Rocha,-34.4833,-54.3333,71.69,47,0,6.51,UY,1616169317
299,Necochea,-38.5473,-58.7368,70.00,51,0,5.01,AR,1616169359
344,Rāzampeta,14.1833,79.1667,75.74,57,0,3.67,IN,1616169383
411,Guerrero,17.6667,-100.0000,78.17,47,0,4.50,MX,1616169416


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Hotel DF
hotel_df = wcriteria_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
91,Pisco,PE,-13.7000,-76.2167,
154,Chui,UY,-33.6971,-53.4616,
184,Honāvar,IN,14.2833,74.4500,
197,Loikaw,MM,19.6742,97.2094,
219,Rocha,UY,-34.4833,-54.3333,
299,Necochea,AR,-38.5473,-58.7368,
344,Rāzampeta,IN,14.1833,79.1667,
411,Guerrero,MX,17.6667,-100.0000,


In [11]:
#Parameters for google search.
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = ["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Results: {index}: {city_name}")
    response = requests.get(url, params=params).json()

    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
      
    except (KeyError, IndexError):
          print(f"Missing result..")
      
print(f"End of search.")
    

Results: 91: ['City']
Closest hotel in ['City'] is Hostal Tambo Colorado.
Results: 154: ['City']
Closest hotel in ['City'] is Descanso al Paso Chuy.
Results: 184: ['City']
Closest hotel in ['City'] is Hotel Sagar Residency.
Results: 197: ['City']
Closest hotel in ['City'] is Kayah Land Hotel.
Results: 219: ['City']
Missing result..
Results: 299: ['City']
Closest hotel in ['City'] is Hotel Mirasol.
Results: 344: ['City']
Closest hotel in ['City'] is DRS RESIDENCY.
Results: 411: ['City']
Missing result..
End of search.


In [12]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
91,Pisco,PE,-13.7000,-76.2167,Hostal Tambo Colorado
154,Chui,UY,-33.6971,-53.4616,Descanso al Paso Chuy
184,Honāvar,IN,14.2833,74.4500,Hotel Sagar Residency
197,Loikaw,MM,19.6742,97.2094,Kayah Land Hotel
219,Rocha,UY,-34.4833,-54.3333,
299,Necochea,AR,-38.5473,-58.7368,Hotel Mirasol
344,Rāzampeta,IN,14.1833,79.1667,DRS RESIDENCY
411,Guerrero,MX,17.6667,-100.0000,


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
fig = gmaps.figure(zoom_level=1.5, center=(20,30))
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

heat_layer = gmaps.heatmap_layer(lat_long, weights=humidity, dissipating=False, max_intensity=100, point_radius=1.5)
fig.add_layer(heat_layer)

# Display figure
fig

IndexError: list index out of range